# U.S. Opiate Prescriptions

Accidental death by fatal drug overdose is a rising trend in the United States. What can you do to help?

This dataset contains summaries of prescription records for 250 common opioid and non-opioid drugs written by 25,000 unique licensed medical professionals in 2014 in the United States for citizens covered under Class D Medicare as well as some metadata about the doctors themselves. This is a small subset of data that was sourced from cms.gov. The full dataset contains almost 24 million prescription instances in long format. I have cleaned and compiled this data here in a format with 1 row per prescriber and limited the approximately 1 million total unique prescribers down to 25,000 to keep it manageable. If you are interested in more data, you can get the script I used to assemble the dataset here and run it yourself. The main data is in prescriber-info.csv. There is also opioids.csv that contains the names of all opioid drugs included in the data and overdoses.csv that contains information on opioid related drug overdose fatalities.

### A key for reading the dataset

NPI – unique National Provider Identifier number
Gender - (M/F)
State - US State by abbreviation
Credentials - set of initials indicative of medical degree
Specialty - description of type of medicinal practice
Opioid.Prescriber - a boolean label indicating whether or not that individual prescribed opiate drugs more than 10 times in the year

In [223]:
import pandas as pd
import numpy as np
import re

In [248]:
#Search for any missing values
missing_values = ['n/a', 'na', '--']

# Read in datasets
opioids = pd.read_csv('opioids.csv', na_values = missing_values)
overdoses = pd.read_csv('overdoses.csv', na_values = missing_values, thousands= ',')
prescriber = pd.read_csv('prescriber-info.csv', na_values = missing_values, thousands= ',')

# Exploratory Analysis

### Prescribers Data Descriptoin

In [246]:
overdoses.head()

,State,Population,Deaths,Abbrev
0,Alabama,4833722,723,AL
1,Alaska,735132,124,AK
2,Arizona,6626624,1211,AZ
3,Arkansas,2959373,356,AR
4,California,38332521,4521,CA


In [239]:
overdoses.describe()

,State,Population,Deaths,Abbrev
count,50,50,50,50
unique,50,50,48,50
top,Florida,"3,850,568",125,MA
freq,1,1,2,1


In [247]:
overdoses['DeathRatio'] = overdoses.Deaths/overdoses.Population
overdoses.head()

,State,Population,Deaths,Abbrev,DeathRatio
0,Alabama,4833722,723,AL,0.000150
1,Alaska,735132,124,AK,0.000169
2,Arizona,6626624,1211,AZ,0.000183
3,Arkansas,2959373,356,AR,0.000120
4,California,38332521,4521,CA,0.000118


In [249]:
prescriber.head()

,NPI,Gender,State,Credentials,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,...,VERAPAMIL.ER,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber
0,1710982582,M,TX,DDS,Dentist,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1245278100,F,AL,MD,General Surgery,0,0,0,0,0,...,0,0,0,0,0,0,0,0,35,1
2,1427182161,F,NY,M.D.,General Practice,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
3,1669567541,M,AZ,MD,Internal Medicine,0,43,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1679650949,M,NV,M.D.,Hematology/Oncology,0,0,0,0,0,...,0,0,0,0,17,28,0,0,0,1


In [77]:
prescriber.describe()

,NPI,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,ALENDRONATE.SODIUM,ALLOPURINOL,ALPRAZOLAM,AMIODARONE.HCL,...,VERAPAMIL.ER,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber
count,2.500000e+04,25000.000000,25000.000000,25000.00000,25000.000000,25000.000000,25000.000000,25000.000000,25000.00000,25000.000000,...,25000.00000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,1.498162e+09,3.157160,2.370400,1.05368,7.041000,0.708440,8.962840,9.305720,16.12812,2.394840,...,2.18804,2.238160,2.551600,0.806240,19.198040,3.348240,4.070880,1.119440,11.573120,0.587520
std,2.877233e+08,20.755819,11.631601,6.66110,25.898648,5.224049,36.520987,29.375476,69.35317,14.266314,...,10.08554,13.325884,23.327117,4.951929,86.501125,17.127556,18.467437,10.299475,39.136616,0.492291
min,1.003002e+09,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.245473e+09,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.497842e+09,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.740406e+09,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.992999e+09,770.000000,644.000000,356.00000,1105.000000,275.000000,2431.000000,790.000000,5716.00000,596.000000,...,528.00000,902.000000,1814.000000,143.000000,6029.000000,596.000000,772.000000,694.000000,2107.000000,1.000000


In [257]:
# Check for any missing data
print(prescribers[prescribers.isnull()])

       NPI Gender Abbrev Credentials Specialty  ABILIFY  \
0      NaN    NaN    NaN         NaN       NaN      NaN   
1      NaN    NaN    NaN         NaN       NaN      NaN   
2      NaN    NaN    NaN         NaN       NaN      NaN   
3      NaN    NaN    NaN         NaN       NaN      NaN   
4      NaN    NaN    NaN         NaN       NaN      NaN   
5      NaN    NaN    NaN         NaN       NaN      NaN   
6      NaN    NaN    NaN         NaN       NaN      NaN   
7      NaN    NaN    NaN         NaN       NaN      NaN   
8      NaN    NaN    NaN         NaN       NaN      NaN   
9      NaN    NaN    NaN         NaN       NaN      NaN   
10     NaN    NaN    NaN         NaN       NaN      NaN   
11     NaN    NaN    NaN         NaN       NaN      NaN   
12     NaN    NaN    NaN         NaN       NaN      NaN   
13     NaN    NaN    NaN         NaN       NaN      NaN   
14     NaN    NaN    NaN         NaN       NaN      NaN   
15     NaN    NaN    NaN         NaN       NaN      NaN 

In [10]:
prescriber.head(5)

,NPI,Gender,State,Credentials,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,...,VERAPAMIL.ER,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber
0,1710982582,M,TX,DDS,Dentist,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1245278100,F,AL,MD,General Surgery,0,0,0,0,0,...,0,0,0,0,0,0,0,0,35,1
2,1427182161,F,NY,M.D.,General Practice,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
3,1669567541,M,AZ,MD,Internal Medicine,0,43,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1679650949,M,NV,M.D.,Hematology/Oncology,0,0,0,0,0,...,0,0,0,0,17,28,0,0,0,1


In [11]:
#Prepare datasets for merge
overdoses_merge = overdoses[['Abbrev', 'Population', 'Deaths']]
overdoses_merge.head()

,Abbrev,Population,Deaths
0,AL,"4,833,722",723
1,AK,"735,132",124
2,AZ,"6,626,624","1,211"
3,AR,"2,959,373",356
4,CA,"38,332,521","4,521"


In [171]:
#Rename 'State' column in Prescriber to 'Abbrev' to prepare dataset for merge
prescriber.rename(columns={'State': 'Abbrev'}, inplace=True)

In [230]:
#Merge dataset into one dataset
###results = pd.merge(prescriber, overdoses_merge, on='Abbrev')###

#Replace hyphens and spaces with periods to match the dataset
prescribers = prescribers.replace(["\ |-","." ], '.')
prescribers.head()

,NPI,Gender,Abbrev,Credentials,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,...,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber,Population,Deaths
0,1710982582,M,TX,DDS,Dentist,0,0,0,0,0,...,0,0,0,0,0,0,0,1,"26,448,193","2,601"
1,1023260569,M,TX,O.D.,Optometry,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"26,448,193","2,601"
2,1659334472,M,TX,MD,General Surgery,0,0,0,0,0,...,0,0,0,0,0,0,0,1,"26,448,193","2,601"
3,1952598419,F,TX,MD,Hematology/Oncology,0,0,16,0,0,...,0,0,40,54,0,0,12,1,"26,448,193","2,601"
4,1043399322,M,TX,MD,Urology,0,23,0,0,0,...,0,0,0,0,0,0,0,1,"26,448,193","2,601"


In [213]:
prescribers.describe()

,NPI,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,ALENDRONATE.SODIUM,ALLOPURINOL,ALPRAZOLAM,AMIODARONE.HCL,...,VERAPAMIL.ER,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber
count,2.468000e+04,24680.000000,24680.000000,24680.000000,24680.000000,24680.000000,24680.000000,24680.000000,24680.000000,24680.000000,...,24680.000000,24680.000000,24680.000000,24680.000000,24680.000000,24680.000000,24680.000000,24680.000000,24680.000000,24680.000000
mean,1.498073e+09,3.171556,2.362844,1.059684,7.090357,0.713047,8.831280,9.311791,16.129092,2.405632,...,2.050567,2.266329,2.541896,0.790924,19.333185,3.347731,4.065640,1.124919,11.413493,0.587642
std,2.877215e+08,20.842807,11.589872,6.670509,26.033903,5.251277,36.221884,29.160535,69.555041,14.338168,...,8.374372,13.409025,23.308381,4.815899,86.988458,17.137469,18.496675,10.345602,36.192726,0.492269
min,1.003002e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.245484e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.497834e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.740412e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.992999e+09,770.000000,644.000000,356.000000,1105.000000,275.000000,2431.000000,790.000000,5716.000000,596.000000,...,169.000000,902.000000,1814.000000,143.000000,6029.000000,596.000000,772.000000,694.000000,1707.000000,1.000000


In [173]:
###for Deaths, Population in results.groupby('Abbrev'):
    print(Deaths)
    print(Population)###

AK
              NPI Gender Abbrev Credentials                     Specialty  \
24076  1740289222      M     AK          MD            Orthopedic Surgery   
24077  1427150549      F     AK        O.D.                     Optometry   
24078  1508972043      M     AK        PA-C           Physician Assistant   
24079  1124005731      M     AK          MD               Family Practice   
24080  1265436752      F     AK      ANP,BC            Nurse Practitioner   
24081  1194980680      M     AK        D.O.             Internal Medicine   
24082  1316093255      F     AK         DMD                       Dentist   
24083  1316019474      M     AK        M.D.               Family Practice   
24084  1316168180      M     AK          MD               Family Practice   
24085  1013003938      M     AK          MD                    Psychiatry   
24086  1841566684      F     AK        PA-C           Physician Assistant   
24087  1235131392      M     AK         DDS  Oral Surgery (dentists only)

[344 rows x 258 columns]
AR
              NPI Gender Abbrev Credentials  \
19546  1164544045      M     AR        D.O.   
19547  1598779209      M     AR        M.D.   
19548  1992771174      M     AR          MD   
19549  1487634432      M     AR        M.D.   
19550  1114359379      F     AR         ANP   
19551  1083610505      M     AR          MD   
19552  1215970561      M     AR          MD   
19553  1578512646      M     AR        M.D.   
19554  1568442366      M     AR        O.D.   
19555  1043265655      M     AR        M.D.   
19556  1184613929      M     AR          MD   
19557  1043288202      M     AR        O.D.   
19558  1942398433      M     AR        M.D.   
19559  1326296658      F     AR         APN   
19560  1982823225      M     AR          MD   
19561  1013281609      M     AR        P.A.   
19562  1902245889      F     AR         M.D   
19563  1790814101      M     AR        M.D.   
19564  1508956079      M     AR          MD   
19565  1740528546      F     AR 

             NPI Gender Abbrev Credentials                          Specialty  \
3800  1669567541      M     AZ          MD                  Internal Medicine   
3801  1609831783      F     AZ      D.D.S.                            Dentist   
3802  1528075702      M     AZ          MD                      Ophthalmology   
3803  1700874153      F     AZ        M.D.                    Family Practice   
3804  1588873715      M     AZ        M.D.                   Thoracic Surgery   
3805  1326090796      M     AZ          MD                 Emergency Medicine   
3806  1710095401      M     AZ         DDS                            Dentist   
3807  1033319322      M     AZ         M.D             Psychiatry & Neurology   
3808  1053500975      M     AZ      D.D.S.                            Dentist   
3809  1245302397      M     AZ      D.D.S.                            Dentist   
3810  1790758662      M     AZ          MD                   Gastroenterology   
3811  1083795215      F     

             NPI Gender Abbrev    Credentials               Specialty  \
6292  1144205303      M     CO             MD         Family Practice   
6293  1508170895      F     CO            DDS                 Dentist   
6294  1629073598      M     CO           M.D.         Family Practice   
6295  1821282682      F     CO             PA     Physician Assistant   
6296  1447444807      F     CO             MD         Family Practice   
6297  1962723130      F     CO           PA-C     Physician Assistant   
6298  1467786418      F     CO            NaN     Physician Assistant   
6299  1306129697      F     CO             PA     Physician Assistant   
6300  1821062043      F     CO            ANP      Nurse Practitioner   
6301  1396779096      M     CO           P.A.     Physician Assistant   
6302  1588702468      F     CO             NP      Nurse Practitioner   
6303  1033109806      F     CO            FNP      Nurse Practitioner   
6304  1184064917      F     CO           M.D.      

              NPI Gender Abbrev       Credentials  \
23091  1477686616      F     DE              M.D.   
23092  1487712196      M     DE               DMD   
23093  1346210887      M     DE                MD   
23094  1144223850      M     DE              M.D.   
23095  1558463299      M     DE                MD   
23096  1861477077      M     DE              M.D.   
23097  1073563987      M     DE              D.O.   
23098  1497820237      F     DE    M.S.N., A.P.N.   
23099  1619182888      F     DE                DO   
23100  1265424170      F     DE                MD   
23101  1649352535      F     DE            PHP MD   
23102  1437475183      F     DE              M.D.   
23103  1861431363      M     DE              D.O.   
23104  1285694935      M     DE              M.D.   
23105  1346240876      M     DE                MD   
23106  1083741599      M     DE              O.D.   
23107  1396748943      F     DE              M.D.   
23108  1629260013      M     DE              M

...

[981 rows x 258 columns]
OK
              NPI Gender Abbrev        Credentials  \
21764  1053405951      F     OK               M.D.   
21765  1447220843      M     OK                 MD   
21766  1265400170      F     OK                 MD   
21767  1609101583      F     OK               ARNP   
21768  1174518336      M     OK              M. D.   
21769  1538189527      F     OK          APRN, CNS   
21770  1194083048      M     OK                NaN   
21771  1316144660      F     OK                 PA   
21772  1558408138      F     OK               PA-C   
21773  1154473254      F     OK                 OD   
21774  1639166499      F     OK  CNP, APRN, CNS-RX   
21775  1598960544      F     OK                 MD   
21776  1487796199      M     OK                 MD   
21777  1346276151      M     OK                DDS   
21778  1417139924      M     OK               D.O.   
21779  1013951409      M     OK                 MD   
21780  1497755540      F     OK               D.O.   


[1211 rows x 258 columns]
RI
              NPI Gender Abbrev  Credentials  \
23760  1972540680      F     RI           MD   
23761  1427019512      F     RI           NP   
23762  1164729083      F     RI           MD   
23763  1972589745      M     RI         M.D.   
23764  1114985041      M     RI         M.D.   
23765  1033205745      M     RI         M.D.   
23766  1932290806      M     RI       D.M.D.   
23767  1063859171      M     RI         M.D.   
23768  1306879242      M     RI           PA   
23769  1831134469      F     RI          PCS   
23770  1083636674      M     RI         M.D.   
23771  1235193707      F     RI         D.O.   
23772  1649359704      F     RI         M.D.   
23773  1780697011      M     RI           PA   
23774  1013071299      M     RI       D.M.D.   
23775  1467546051      M     RI         M.D.   
23776  1376654350      M     RI           MD   
23777  1457333619      M     RI         M.D.   
23778  1083839138      M     RI          NaN   
23779  1861

              NPI Gender Abbrev Credentials  \
24597  1255642526      M     SD          MD   
24598  1457380859      M     SD          MD   
24599  1528068152      M     SD        O.D.   
24600  1336196724      M     SD          MD   
24601  1992725436      M     SD          MD   
24602  1174705115      F     SD        O.D.   
24603  1568690758      M     SD          MD   
24604  1083811632      M     SD         DDS   
24605  1669410650      F     SD        PA-C   
24606  1487616827      F     SD          MD   
24607  1871751867      M     SD          MD   
24608  1881659720      F     SD        PA-C   
24609  1720052228      M     SD        M.D.   
24610  1154582765      F     SD        M.D.   
24611  1821372764      F     SD         CNP   
24612  1396749768      F     SD        M.D.   
24613  1598762544      F     SD          MD   
24614  1235195041      M     SD          MD   
24615  1750396560      F     SD         CNP   
24616  1124074588      F     SD         PAC   
24617  117457

              NPI Gender Abbrev      Credentials  \
10797  1083907059      M     UT             M.D.   
10798  1366495459      F     UT               MD   
10799  1346294717      M     UT             M.D.   
10800  1023213287      M     UT              DMD   
10801  1265601314      M     UT              DDS   
10802  1346239548      M     UT               MD   
10803  1790825768      M     UT               MD   
10804  1285724807      M     UT               MD   
10805  1790861029      F     UT               MD   
10806  1376648840      M     UT              DMD   
10807  1063500163      M     UT             M.D.   
10808  1619177540      M     UT             N.D.   
10809  1548510431      F     UT  DNP, APRN, NP-C   
10810  1366651481      M     UT             M.D.   
10811  1295925642      F     UT             P.A.   
10812  1689861528      F     UT               MD   
10813  1487603916      M     UT             M.D.   
10814  1306805007      M     UT             O.D.   
10815  18718

              NPI Gender Abbrev         Credentials  \
23182  1437186061      M     WA                M.D.   
23183  1770772881      F     WA                ARNP   
23184  1295721322      M     WA                  MD   
23185  1316942287      F     WA              D.D.S.   
23186  1356570923      M     WA                M.D.   
23187  1669646543      M     WA                M.D.   
23188  1295045516      F     WA                ARNP   
23189  1932217726      M     WA                 DDS   
23190  1255381109      M     WA                M.D.   
23191  1699764472      F     WA                M.D.   
23192  1336331636      F     WA                ARNP   
23193  1407067432      F     WA                M.D.   
23194  1821188640      F     WA                  MD   
23195  1194904508      M     WA                 NaN   
23196  1124079140      M     WA                M.D.   
23197  1295790129      F     WA                  MD   
23198  1578771853      M     WA                M.D.   
23199  175

In [175]:
###prescribed_opioids = (list(opioids), set(prescriber.columns))
prescribed_opioids###

(['Drug Name', 'Generic Name'],
 {'ABILIFY',
  'ACETAMINOPHEN.CODEINE',
  'ACYCLOVIR',
  'ADVAIR.DISKUS',
  'AGGRENOX',
  'ALENDRONATE.SODIUM',
  'ALLOPURINOL',
  'ALPRAZOLAM',
  'AMIODARONE.HCL',
  'AMITRIPTYLINE.HCL',
  'AMLODIPINE.BESYLATE',
  'AMLODIPINE.BESYLATE.BENAZEPRIL',
  'AMOX.TR.POTASSIUM.CLAVULANATE',
  'AMOXICILLIN',
  'AMPHETAMINE.SALT.COMBO',
  'ATENOLOL',
  'ATORVASTATIN.CALCIUM',
  'AVODART',
  'AZITHROMYCIN',
  'Abbrev',
  'BACLOFEN',
  'BD.ULTRA.FINE.PEN.NEEDLE',
  'BENAZEPRIL.HCL',
  'BENICAR',
  'BENICAR.HCT',
  'BENZTROPINE.MESYLATE',
  'BISOPROLOL.HYDROCHLOROTHIAZIDE',
  'BRIMONIDINE.TARTRATE',
  'BUMETANIDE',
  'BUPROPION.HCL.SR',
  'BUPROPION.XL',
  'BUSPIRONE.HCL',
  'BYSTOLIC',
  'CARBAMAZEPINE',
  'CARBIDOPA.LEVODOPA',
  'CARISOPRODOL',
  'CARTIA.XT',
  'CARVEDILOL',
  'CEFUROXIME',
  'CELEBREX',
  'CEPHALEXIN',
  'CHLORHEXIDINE.GLUCONATE',
  'CHLORTHALIDONE',
  'CILOSTAZOL',
  'CIPROFLOXACIN.HCL',
  'CITALOPRAM.HBR',
  'CLINDAMYCIN.HCL',
  'CLOBETASOL.PROP

In [176]:
###ggroup = results['Abbrev'].groupby([results['Gender'], results['Deaths']])
ggroup
results.sort_values(by='Specialty')# sort the pandas dataframe by ascending value of single column
results.sort_values(by=['Deaths', 'Gender'],ascending=[True,False]) # sort the pandas dataframe by multiple columns###

,NPI,Gender,Abbrev,Credentials,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,...,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber,Population,Deaths
19063,1609867399,M,MO,M.D.,Internal Medicine,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"6,044,171","1,067"
19064,1578724068,M,MO,D.O.,Internal Medicine,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"6,044,171","1,067"
19065,1649533845,M,MO,M.D.,Internal Medicine,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"6,044,171","1,067"
19066,1831230655,M,MO,MD,Obstetrics/Gynecology,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"6,044,171","1,067"
19067,1891704193,M,MO,DDS,Dentist,0,0,0,0,0,...,0,0,0,0,0,0,0,1,"6,044,171","1,067"
19068,1093742314,M,MO,MD,Obstetrics/Gynecology,0,0,0,0,0,...,0,0,0,0,0,0,0,1,"6,044,171","1,067"
19070,1558419457,M,MO,NaN,Dentist,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"6,044,171","1,067"
19071,1558661116,M,MO,D.O.,Otolaryngology,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"6,044,171","1,067"
19073,1326057407,M,MO,D.D.S.,Dentist,0,0,0,0,0,...,0,0,0,0,0,0,0,1,"6,044,171","1,067"
19074,1093014441,M,MO,M.D.,Student in an Organized Health Care Education/...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,"6,044,171","1,067"


In [97]:
###genderCount = results[['Gender', 'NPI']].groupby('Gender').count()['NPI']
genderCount###

Gender
F     9318
M    15362
Name: NPI, dtype: int64

In [261]:
p = prescribers.loc[:, prescribers.columns.str.contains('DILTIAZEM')]
print(p.head(3))

   DILTIAZEM.24HR.CD  DILTIAZEM.24HR.ER  DILTIAZEM.ER  DILTIAZEM.HCL
0                  0                  0             0              0
1                  0                  0             0              0
2                  0                  0             0              0


In [215]:
prescribers.shape

(24680, 258)

In [216]:
list(prescribers.columns)

['NPI',
 'Gender',
 'Abbrev',
 'Credentials',
 'Specialty',
 'ABILIFY',
 'ACETAMINOPHEN.CODEINE',
 'ACYCLOVIR',
 'ADVAIR.DISKUS',
 'AGGRENOX',
 'ALENDRONATE.SODIUM',
 'ALLOPURINOL',
 'ALPRAZOLAM',
 'AMIODARONE.HCL',
 'AMITRIPTYLINE.HCL',
 'AMLODIPINE.BESYLATE',
 'AMLODIPINE.BESYLATE.BENAZEPRIL',
 'AMOXICILLIN',
 'AMOX.TR.POTASSIUM.CLAVULANATE',
 'AMPHETAMINE.SALT.COMBO',
 'ATENOLOL',
 'ATORVASTATIN.CALCIUM',
 'AVODART',
 'AZITHROMYCIN',
 'BACLOFEN',
 'BD.ULTRA.FINE.PEN.NEEDLE',
 'BENAZEPRIL.HCL',
 'BENICAR',
 'BENICAR.HCT',
 'BENZTROPINE.MESYLATE',
 'BISOPROLOL.HYDROCHLOROTHIAZIDE',
 'BRIMONIDINE.TARTRATE',
 'BUMETANIDE',
 'BUPROPION.HCL.SR',
 'BUPROPION.XL',
 'BUSPIRONE.HCL',
 'BYSTOLIC',
 'CARBAMAZEPINE',
 'CARBIDOPA.LEVODOPA',
 'CARISOPRODOL',
 'CARTIA.XT',
 'CARVEDILOL',
 'CEFUROXIME',
 'CELEBREX',
 'CEPHALEXIN',
 'CHLORHEXIDINE.GLUCONATE',
 'CHLORTHALIDONE',
 'CILOSTAZOL',
 'CIPROFLOXACIN.HCL',
 'CITALOPRAM.HBR',
 'CLINDAMYCIN.HCL',
 'CLOBETASOL.PROPIONATE',
 'CLONAZEPAM',
 'CLONI

In [217]:
###df6 = prescribers.Specialty.value_counts()
df6###

Internal Medicine                                                 3146
Family Practice                                                   2965
Dentist                                                           2770
Nurse Practitioner                                                2505
Physician Assistant                                               1835
Emergency Medicine                                                1076
Psychiatry                                                         682
Cardiology                                                         675
Obstetrics/Gynecology                                              606
Orthopedic Surgery                                                 572
Optometry                                                          570
Student in an Organized Health Care Education/Training Program     532
Ophthalmology                                                      511
General Surgery                                                    484
Gastro

In [291]:
prescriber['DILTIAZEM'] = (prescriber['DILTIAZEM.24HR.CD'] + prescriber['DILTIAZEM.24HR.ER'] + prescriber['DILTIAZEM.ER'] + prescriber['DILTIAZEM.HCL'])

In [292]:
prescriber.columns[50:85]

Index(['CLINDAMYCIN.HCL', 'CLOBETASOL.PROPIONATE', 'CLONAZEPAM',
       'CLONIDINE.HCL', 'CLOPIDOGREL', 'CLOTRIMAZOLE.BETAMETHASONE', 'COLCRYS',
       'COMBIVENT.RESPIMAT', 'CRESTOR', 'CYCLOBENZAPRINE.HCL', 'DEXILANT',
       'DIAZEPAM', 'DICLOFENAC.SODIUM', 'DICYCLOMINE.HCL', 'DIGOX', 'DIGOXIN',
       'DILTIAZEM.24HR.CD', 'DILTIAZEM.24HR.ER', 'DILTIAZEM.ER',
       'DILTIAZEM.HCL', 'DIOVAN', 'DIPHENOXYLATE.ATROPINE',
       'DIVALPROEX.SODIUM', 'DIVALPROEX.SODIUM.ER', 'DONEPEZIL.HCL',
       'DORZOLAMIDE.TIMOLOL', 'DOXAZOSIN.MESYLATE', 'DOXEPIN.HCL',
       'DOXYCYCLINE.HYCLATE', 'DULOXETINE.HCL', 'ENALAPRIL.MALEATE',
       'ESCITALOPRAM.OXALATE', 'ESTRADIOL', 'EXELON', 'FAMOTIDINE'],
      dtype='object')

In [294]:
prescriber.iloc[50:85]

,NPI,Gender,State,Credentials,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,...,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber,DILTIAZEM
50,1235426800,F,PA,CRNP,Nurse Practitioner,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
51,1902866106,F,LA,M.D.,Obstetrics/Gynecology,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52,1952353724,M,MD,M.D.,Cardiology,0,0,0,0,0,...,0,0,0,48,26,58,0,0,0,12
53,1043399322,M,TX,MD,Urology,0,23,0,0,0,...,82,0,0,0,0,0,0,0,1,0
54,1255471140,M,IL,MD,Cardiac Electrophysiology,0,0,0,0,0,...,0,0,0,56,36,0,0,0,0,50
55,1619975919,F,CT,PAC,Physician Assistant,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
56,1023062759,M,MI,M.D.,Family Practice,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
57,1376865949,F,PR,D.M.D.,Dentist,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58,1609867399,M,MO,M.D.,Internal Medicine,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59,1225079817,F,IA,M.D.,Family Practice,0,0,0,0,0,...,0,0,0,24,0,0,0,0,1,0
